**Installation and imports**

In [ ]:
!pip install onnxruntime

import os
import json
import zipfile
import numpy as np
import tensorflow as tf
from keras import utils
import onnxruntime as ort
from datetime import datetime
from sklearn.metrics import accuracy_score, classification_report

**Define unzip function**

In [ ]:
def unzip_file(zip_file_path, extract_to_dir):
    if not os.path.exists(extract_to_dir):
        os.makedirs(extract_to_dir)

    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_dir)

    print(f"Files extracted to: {extract_to_dir}")

**Unzip images and models**

In [ ]:
zip_file_path = 'images_zipped.zip'
extract_to_dir = 'train'
unzip_file(zip_file_path, extract_to_dir)

In [ ]:
zip_file_path = 'models.zip'
extract_to_dir = 'models'
unzip_file(zip_file_path, extract_to_dir)

**Create validation set**

In [ ]:
path_to_data = 'train/'

In [ ]:
val_data=utils.image_dataset_from_directory(
    path_to_data,
    labels="inferred",
    label_mode="int",
    validation_split=0.1,
    subset="validation",
    color_mode="rgb",
    image_size=(256,256),
    batch_size=64,
    seed=40,
)

**Verify validation set with labels (classes)**

In [ ]:
for images,lables in val_data.take(1):
  print('Amount of train images: ', images.shape)
  print('Amount of train labels: ', lables.shape)

**Define normalization function and normalize validation set**

In [ ]:
def normalize(image, label):
  return image/255.0, label

In [ ]:
val_data= val_data.map(normalize)

**Create one Tensor of the validation images and labels**

In [ ]:
val_x=[]
val_y=[]

for image,label in val_data:
   val_x.append(image)
   val_y.append(label)

val_x = tf.concat(val_x, axis=0)
val_y = tf.concat(val_y, axis=0)

**Create binary matrix representation of the labels (One-Hot Encoding format)**

In [ ]:
num_classes = 10
val_y = tf.keras.utils.to_categorical(val_y, num_classes=num_classes)

**Define function to get the name of the model file**

In [ ]:
def find_model_file(folder_path):
    for file in os.listdir(folder_path):
        if file.endswith('.onnx'):
            return file
    return None

**Get model file names**

In [ ]:
new_model_name = find_model_file('modals/newmodal')
production_model_name = find_model_file('modals/production')

**Create inference sessions for both the onnx models and retrieve input and outputs**

In [ ]:
new_model = f"modals/newmodal/{new_model_name}"
new_model_session = ort.InferenceSession(new_model)

production_model = f"modals/production/{production_model_name}"
production_model_session = ort.InferenceSession(production_model)

new_model_input = new_model_session.get_inputs()[0].name
new_model_output = new_model_session.get_outputs()[0].name

production_model_input = production_model_session.get_inputs()[0].name
production_model_output = production_model_session.get_outputs()[0].name

**Iterate through validation set and get predictions from both models**

In [ ]:
new_pred_arr = []
production_pred_arr = []

for i, val in enumerate(val_x):
    new_model_pred = new_model_session.run([new_model_output], {new_model_input: [val_x[i].cpu().numpy()]})
    production_model_pred = production_model_session.run([production_model_output], {production_model_input: [val_x[i].cpu().numpy()]})

    new_pred_arr.append(new_model_pred)
    production_pred_arr.append(production_model_pred)

**Retrieve accuracy (amount of correct predictions of the set) for both models**

In [ ]:
new_pred_arr = np.concatenate([np.argmax(pred[0], axis=1) for pred in new_pred_arr], axis=0)
production_pred_arr = np.concatenate([np.argmax(pred[0], axis=1) for pred in production_pred_arr], axis=0)

val_y_flat = np.argmax(val_y, axis=1)

new_model_accuracy = accuracy_score(val_y_flat, new_pred_arr)
production_model_accuracy = accuracy_score(val_y_flat, production_pred_arr)

**Add to .json file if new model is better than production model**

In [ ]:
should_replace = new_model_accuracy > production_model_accuracy

with open('should_replace.json', 'w') as f:
    json.dump({'replace': should_replace}, f)

**Zip .json file and, if better, zip new model with it**

In [ ]:
new_model_folder = 'modals/newmodal'
replacement_zip = 'replacement.zip'

with zipfile.ZipFile(replacement_zip, 'w') as zipf:
    zipf.write('should_replace.json')

    if should_replace:
        for root, dirs, files in os.walk(new_model_folder):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, new_model_folder)
                zipf.write(file_path, arcname)

print(f"Files zipped into '{replacement_zip}'.")


**Write run information, accuracy's and classification reports to text file, which will produce a pipeline run artefact**

In [ ]:
filename = 'model_comparison.txt'

with open(filename, 'w') as file:
    file.write(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
    file.write(f"New Model Name: {new_model_name}\n")
    file.write(f"New Model Accuracy: {new_model_accuracy * 100:.2f}%\n\n")
    file.write(f"Production Model Name: {production_model_name}\n")
    file.write(f"Production Model Accuracy: {production_model_accuracy * 100:.2f}%\n")

    file.write("\nNew Model Classification Report:\n")
    file.write(classification_report(val_y_flat, new_pred_arr, zero_division=0))

    file.write("\nProduction Model Classification Report:\n")
    file.write(classification_report(val_y_flat, production_pred_arr, zero_division=0))

print(f"Results saved to {filename}")